In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder



In [2]:
# Load the data
file_path=("Resources/IRL_ClassProjectData_220210.xlsx")
fairtrade_df=pd.read_excel(file_path)

fairtrade_df.head(10)

,Survey Number,Response ID,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,understand_contract,entity_tenure - Months,entity_seasons,...,unsafe_reason_ms_core,accident_last_year,injury_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,leadership_satisfaction_core,factory_recommend_tf_core
0,Survey 2,R_1NsRph9Oqxl6CnG,Farm 1,Man,30,Elementary School,Permanent,Yes,132.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,Yes,No,NaN,7
1,Survey 2,R_2BlpeLG3DymUC2o,Farm 1,Woman,30,No school,Permanent,Yes,108.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,8
2,Survey 2,R_3WMv7jb7gBKEm2t,Farm 1,Man,24,Middle School,Permanent,Yes,67.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,8
3,Survey 2,R_2SBT3CXWOyuor1k,Farm 1,Man,58,Elementary School,Permanent,Yes,79.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,6
4,Survey 2,R_OBEgl4lS6LpSkDL,Farm 1,Man,38,Middle School,Permanent,Yes,108.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,8
5,Survey 2,R_3QY8IjWLk2ZDKDs,Farm 1,Man,37,Elementary School,Permanent,Yes,108.0,NaN,...,NaN,No,NaN,Strongly Agree,Don't know,Strongly disagree,Don't know,No,NaN,8
6,Survey 2,R_1jHPFIUCc8JCpXa,Farm 1,Woman,27,Middle School,Permanent,Yes,33.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,NaN,7
7,Survey 2,R_6x0zO9KWBj8KUDL,Farm 1,Man,56,Elementary School,Permanent,Don't know,464.0,NaN,...,NaN,No,NaN,Somewhat agree,Yes,Agree nor disagree,Don't know,No,NaN,5 - Neutral
8,Survey 2,R_cMGCFYIGxSURpzX,Farm 1,Woman,45,No school,Permanent,Yes,84.0,NaN,...,NaN,No,NaN,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,NaN,8
9,Survey 2,R_1q8QVbp49i6YbKB,Farm 1,Woman,28,Elementary School,Permanent,Yes,84.0,NaN,...,NaN,No,NaN,Strongly Agree,Yes,Strongly disagree,Yes,No,NaN,10 - Extremely likely


In [3]:
# Change column names 
fairtrade_df.rename(columns = {'entity_tenure - Months':'entity_tenure_total_months'}, inplace = True)

# # Drop columns 
# fairtrade_df.drop(['Response ID','gender_core','return_next_season_core','age_core','work_last_season_tf','num_week_income_core_usd',
#                    'num_week_income_core_mxn','weeks_per_year','income_change_core','save_money_tf_core',
#                    'financial_resilience_core','unsafe_reason_ms_core','management_complaint_treatment_core',
#                    'leadership_feedback_core','leadership_satisfaction_core','injury_last_year','understand_contract', 'entity_seasons','trust_leadership_core'], axis=1, inplace=True)


# # Drop columns TESTING 
fairtrade_df.drop(['Response ID','work_last_season_tf','num_week_income_core_usd',
                   'num_week_income_core_mxn','weeks_per_year','income_change_core','save_money_tf_core',
                   'financial_resilience_core','unsafe_reason_ms_core','entity_seasons',
                   'leadership_satisfaction_core','injury_last_year','understand_contract','trust_leadership_core'], axis=1, inplace=True)
fairtrade_df.head(10)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132.0,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,8
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,7
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464.0,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,5 - Neutral
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84.0,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,8
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84.0,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10 - Extremely likely


In [4]:
# fairtrade_df['workplace_satis_core'].unique()
# fairtrade_df['conditions_rate_core'].unique()

# Identified that Strongly Agree and Strongly agree were spelled slightly different
fairtrade_df['safety_core'].value_counts()
# fairtrade_df['safety_core'].unique()

# fairtrade_df['covid_safety_measures_core'].unique()
# fairtrade_df['covid_safety_measures_core'].value_counts()
# Identified that Strongly Agree and Strongly agree had 2 separate instances.  Also Agree nor Disagree had 2 instances
# fairtrade_df['pressure_to_work_core'].unique()
# fairtrade_df['pressure_to_work_core'].value_counts()



Strongly Agree        2137
Strongly agree        1427
Somewhat agree         639
Agree nor disagree     343
Somewhat disagree      109
Strongly disagree      100
Refused                 16
Name: safety_core, dtype: int64

In [5]:
# Drop rows that contain  refused or don't know

values = ["Refused", "Don't know"]
fairtrade_df = fairtrade_df[fairtrade_df.factory_recommend_tf_core.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.accident_last_year.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.workplace_satis_core.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.conditions_rate_core.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.safety_core.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.covid_safety_measures_core.isin(values)==False]
fairtrade_df = fairtrade_df[fairtrade_df.pressure_to_work_core.isin(values)==False]




# Standardize values in columns so that they're consistent across the column
dict= {"10 - Extremely likely ": "10", "0- Extremely unlikely": "0", "5 - Neutral": "5"}
dict_SA= {"Strongly Agree":"Strongly agree"}
dict_Neither={"Neither agree nor disagree":"Agree nor disagree"}
fairtrade_df = fairtrade_df.replace({"factory_recommend_tf_core": dict})
fairtrade_df= fairtrade_df.replace({"safety_core": dict_SA})
fairtrade_df= fairtrade_df.replace({"pressure_to_work_core":dict_Neither})
fairtrade_df= fairtrade_df.replace({"pressure_to_work_core":dict_SA})


fairtrade_df.reset_index(inplace=True, drop=True)

fairtrade_df.head(45)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132.0,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79.0,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108.0,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,8
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33.0,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,7
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464.0,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,5
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84.0,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,8
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84.0,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10


In [6]:
# Identify the null rows
fairtrade_df.isnull().sum()

Survey Number                            0
ch_account_name                          0
gender_core                              0
age_core                                 0
education_level_core                     0
worker_type_core                         0
entity_tenure_total_months               0
workplace_satis_core                     0
living_wage_tf_core                      0
conditions_rate_core                     0
safety_core                              0
accident_last_year                       0
covid_safety_measures_core             222
return_next_season_core                222
pressure_to_work_core                  222
management_complaint_treatment_core    222
leadership_feedback_core               222
factory_recommend_tf_core              222
dtype: int64

In [7]:
# Drop the null rows 
# fairtrade_df.dropna(subset = ['covid_safety_measures_core','return_next_season_core','pressure_to_work_core','factory_recommend_tf_core'], inplace=True)
# fairtrade_df.isnull().sum()

# DELETE AFTER TESTING

fairtrade_df.dropna(subset = ['covid_safety_measures_core','return_next_season_core','leadership_feedback_core','management_complaint_treatment_core','return_next_season_core','pressure_to_work_core','factory_recommend_tf_core'], inplace=True)
fairtrade_df.isnull().sum()

Survey Number                          0
ch_account_name                        0
gender_core                            0
age_core                               0
education_level_core                   0
worker_type_core                       0
entity_tenure_total_months             0
workplace_satis_core                   0
living_wage_tf_core                    0
conditions_rate_core                   0
safety_core                            0
accident_last_year                     0
covid_safety_measures_core             0
return_next_season_core                0
pressure_to_work_core                  0
management_complaint_treatment_core    0
leadership_feedback_core               0
factory_recommend_tf_core              0
dtype: int64

In [8]:
# Check the datatypes of our columns 
fairtrade_df.dtypes

Survey Number                           object
ch_account_name                         object
gender_core                             object
age_core                                 int64
education_level_core                    object
worker_type_core                        object
entity_tenure_total_months             float64
workplace_satis_core                    object
living_wage_tf_core                     object
conditions_rate_core                    object
safety_core                             object
accident_last_year                      object
covid_safety_measures_core              object
return_next_season_core                 object
pressure_to_work_core                   object
management_complaint_treatment_core     object
leadership_feedback_core                object
factory_recommend_tf_core               object
dtype: object

In [9]:
# Convert columns to numeric data type
fairtrade_df[["entity_tenure_total_months", "factory_recommend_tf_core"]] = fairtrade_df[["entity_tenure_total_months", "factory_recommend_tf_core"]].apply(pd.to_numeric)
fairtrade_df['entity_tenure_total_months'] = fairtrade_df['entity_tenure_total_months'].astype(int)

fairtrade_df.dtypes

Survey Number                          object
ch_account_name                        object
gender_core                            object
age_core                                int64
education_level_core                   object
worker_type_core                       object
entity_tenure_total_months              int64
workplace_satis_core                   object
living_wage_tf_core                    object
conditions_rate_core                   object
safety_core                            object
accident_last_year                     object
covid_safety_measures_core             object
return_next_season_core                object
pressure_to_work_core                  object
management_complaint_treatment_core    object
leadership_feedback_core               object
factory_recommend_tf_core               int64
dtype: object

In [10]:
# Transform factory_recommend_tf_core to have positive or negative status
# Values 0-5 would be coded as '0' (Negative) 
# Values 6-10 would be coded as '1' (Positive)

fairtrade_df['factory_recommend_tf_score'] = pd.cut(fairtrade_df['factory_recommend_tf_core'], bins=[0, 5, 10], include_lowest=True, labels=['0', '1'])

fairtrade_df.head(20)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_core,factory_recommend_tf_score
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,7,1
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8,1
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8,1
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,6,1
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,8,1
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,8,1
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,7,1
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,5,0
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,8,1
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,10,1


In [11]:
# Drop old factory_recommend_tf_core
fairtrade_df.drop(['factory_recommend_tf_core'], axis=1, inplace=True)

fairtrade_df.head(20)

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,workplace_satis_core,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_score
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132,Somewhat Satisfied,No,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,1
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79,Very satisfied,Yes,Somewhat good,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
5,Survey 2,Farm 1,Man,37,Elementary School,Permanent,108,Somewhat Satisfied,No,Neither good or bad,Strongly agree,No,Strongly Agree,Don't know,Strongly disagree,Don't know,No,1
6,Survey 2,Farm 1,Woman,27,Middle School,Permanent,33,Very satisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,"Maybe, depending on the topic",No,1
7,Survey 2,Farm 1,Man,56,Elementary School,Permanent,464,Somewhat Satisfied,No,Neither good or bad,Somewhat agree,No,Somewhat agree,Yes,Agree nor disagree,Don't know,No,0
8,Survey 2,Farm 1,Woman,45,No school,Permanent,84,Very satisfied,No,Somewhat good,Strongly agree,No,Somewhat agree,Yes,Agree nor disagree,"Maybe, depending on the topic",No,1
9,Survey 2,Farm 1,Woman,28,Elementary School,Permanent,84,Neither satisfied nor dissatisfied,Yes,Neither good or bad,Strongly agree,No,Strongly Agree,Yes,Strongly disagree,Yes,No,1


In [12]:
fairtrade_df.shape

(4279, 18)

# Split the Data into Training and Testing

In [13]:
# Ordinal Encode Columns

# Create copy of Dataframe before Encoding
test_df=fairtrade_df.copy()

# Create dictionaries for each column *We can verify later if some of these are redudant
dict_1={'Very dissatisfied':1,'Somewhat dissatisfied':2, 'Neither satisfied nor dissatisfied':3, 'Somewhat Satisfied':4,'Very satisfied':5}
dict_2={'Somewhat good':4, 'Neither good or bad':3, 'Very good':5,
       'Somewhat bad':2, 'Very bad':1}
dict_3={'Strongly agree':5, 'Somewhat agree':4, 'Agree nor disagree':3,
       'Somewhat disagree':2, 'Strongly disagree':1}
dict_4={'Strongly Agree': 5, 'Somewhat agree':4, 'Agree nor disagree':3,
       'Strongly disagree':1, 'Somewhat disagree':2}
dict_5={'Strongly disagree':1, 'Agree nor disagree':3, 'Strongly agree':5,
       'Somewhat disagree':2, 'Somewhat agree':4}

#Use map function to encode strings with ordinal numbers
test_df['workplace_satis_score']=test_df.workplace_satis_core.map(dict_1)
test_df['conditions_rate_core']=test_df.conditions_rate_core.map(dict_2)
test_df['safety_core']=test_df.safety_core.map(dict_3)
test_df['covid_safety_measures_core']=test_df.covid_safety_measures_core.map(dict_4)
test_df['pressure_to_work_core']=test_df.pressure_to_work_core.map(dict_5)

# test_df[['workplace_satis_core','workplace_satis_score','conditions_rate_core','safety_core','covid_safety_measures_core','pressure_to_work_core']]
test_df.drop(['workplace_satis_core'], axis=1, inplace=True)
test_df.head()

,Survey Number,ch_account_name,gender_core,age_core,education_level_core,worker_type_core,entity_tenure_total_months,living_wage_tf_core,conditions_rate_core,safety_core,accident_last_year,covid_safety_measures_core,return_next_season_core,pressure_to_work_core,management_complaint_treatment_core,leadership_feedback_core,factory_recommend_tf_score,workplace_satis_score
0,Survey 2,Farm 1,Man,30,Elementary School,Permanent,132,No,4,5,No,5,Yes,1,Yes,No,1,4
1,Survey 2,Farm 1,Woman,30,No school,Permanent,108,No,3,5,No,5,Yes,1,"Maybe, depending on the topic",No,1,4
2,Survey 2,Farm 1,Man,24,Middle School,Permanent,67,Yes,4,5,No,5,Yes,1,"Maybe, depending on the topic",No,1,5
3,Survey 2,Farm 1,Man,58,Elementary School,Permanent,79,Yes,4,5,No,5,Yes,1,"Maybe, depending on the topic",No,1,5
4,Survey 2,Farm 1,Man,38,Middle School,Permanent,108,Yes,3,5,No,5,Yes,1,"Maybe, depending on the topic",No,1,5


In [14]:
# Create our features

#Indicate features that we will exclude from model
columns2drop=['factory_recommend_tf_score','worker_type_core', 'leadership_feedback_core', 'gender_core', 'worker_type_core', 'education_level_core','leadership_feedback_core', 'management_complaint_treatment_core', 'accident_last_year', 'Survey Number','living_wage_tf_core','leadership_feedback_core',
             'return_next_season_core']
column2drop=['factory_recommend_tf_score']
#Create our
X=pd.get_dummies(test_df.drop(columns2drop, axis=1))

# Create our target
y=test_df['factory_recommend_tf_score']

X

,age_core,entity_tenure_total_months,conditions_rate_core,safety_core,covid_safety_measures_core,pressure_to_work_core,workplace_satis_score,ch_account_name_Farm 1,ch_account_name_Farm 10,ch_account_name_Farm 11 \t\t\t\t\t\t\t,ch_account_name_Farm 12,ch_account_name_Farm 2,ch_account_name_Farm 3,ch_account_name_Farm 4,ch_account_name_Farm 5,ch_account_name_Farm 6,ch_account_name_Farm 7,ch_account_name_Farm 8,ch_account_name_Farm 9
0,30,132,4,5,5,1,4,1,0,0,0,0,0,0,0,0,0,0,0
1,30,108,3,5,5,1,4,1,0,0,0,0,0,0,0,0,0,0,0
2,24,67,4,5,5,1,5,1,0,0,0,0,0,0,0,0,0,0,0
3,58,79,4,5,5,1,5,1,0,0,0,0,0,0,0,0,0,0,0
4,38,108,3,5,5,1,5,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,41,240,4,5,5,1,5,0,0,0,1,0,0,0,0,0,0,0,0
4497,36,156,5,5,5,1,5,0,0,0,1,0,0,0,0,0,0,0,0
4498,37,144,5,5,5,1,5,0,0,0,1,0,0,0,0,0,0,0,0
4499,22,1,5,1,5,1,4,0,0,0,1,0,0,0,0,0,0,0,0


In [15]:
X.describe()
X

,age_core,entity_tenure_total_months,conditions_rate_core,safety_core,covid_safety_measures_core,pressure_to_work_core,workplace_satis_score,ch_account_name_Farm 1,ch_account_name_Farm 10,ch_account_name_Farm 11 \t\t\t\t\t\t\t,ch_account_name_Farm 12,ch_account_name_Farm 2,ch_account_name_Farm 3,ch_account_name_Farm 4,ch_account_name_Farm 5,ch_account_name_Farm 6,ch_account_name_Farm 7,ch_account_name_Farm 8,ch_account_name_Farm 9
0,30,132,4,5,5,1,4,1,0,0,0,0,0,0,0,0,0,0,0
1,30,108,3,5,5,1,4,1,0,0,0,0,0,0,0,0,0,0,0
2,24,67,4,5,5,1,5,1,0,0,0,0,0,0,0,0,0,0,0
3,58,79,4,5,5,1,5,1,0,0,0,0,0,0,0,0,0,0,0
4,38,108,3,5,5,1,5,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4496,41,240,4,5,5,1,5,0,0,0,1,0,0,0,0,0,0,0,0
4497,36,156,5,5,5,1,5,0,0,0,1,0,0,0,0,0,0,0,0
4498,37,144,5,5,5,1,5,0,0,0,1,0,0,0,0,0,0,0,0
4499,22,1,5,1,5,1,4,0,0,0,1,0,0,0,0,0,0,0,0


In [16]:
y.describe()

count     4279
unique       2
top          1
freq      3864
Name: factory_recommend_tf_score, dtype: object

In [17]:
# Check the value count for y outcomes
#0 is demoter
#1 is promoter

y.value_counts()

1    3864
0     415
Name: factory_recommend_tf_score, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=1,
                                                   stratify=y)

X_train.shape

(3209, 19)

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
clf = BalancedRandomForestClassifier(random_state=0, n_estimators=100)
clf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=0)

In [20]:
y_pred = clf.predict(X_test)
y_pred

array(['1', '1', '1', ..., '1', '1', '1'], dtype=object)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7497412008281574

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)



array([[ 78,  26],
       [242, 724]])

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.24      0.75      0.75      0.37      0.75      0.56       104
          1       0.97      0.75      0.75      0.84      0.75      0.56       966

avg / total       0.90      0.75      0.75      0.80      0.75      0.56      1070



In [24]:
# List the features sorted in descending order by feature importance
feature_importances = clf.feature_importances_
features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1], reverse=True)
features

[('age_core', 0.16675036619190037),
 ('entity_tenure_total_months', 0.16245189453720155),
 ('conditions_rate_core', 0.11864797211844326),
 ('pressure_to_work_core', 0.09499701917188783),
 ('workplace_satis_score', 0.09050696715103548),
 ('safety_core', 0.08670018719650802),
 ('covid_safety_measures_core', 0.08391300886395785),
 ('ch_account_name_Farm 3', 0.04371561822615308),
 ('ch_account_name_Farm 5', 0.02552839723600252),
 ('ch_account_name_Farm 4', 0.020805239454313097),
 ('ch_account_name_Farm 7', 0.018131238011901505),
 ('ch_account_name_Farm 10', 0.016093122665906553),
 ('ch_account_name_Farm 6', 0.013122868091325707),
 ('ch_account_name_Farm 2', 0.012954992231867534),
 ('ch_account_name_Farm 12', 0.01149409529005397),
 ('ch_account_name_Farm 11  \t\t\t\t\t\t\t', 0.01083568561154743),
 ('ch_account_name_Farm 1', 0.009107973030209939),
 ('ch_account_name_Farm 9', 0.007765706764152861),
 ('ch_account_name_Farm 8', 0.006477648155631484)]